# Extract From HKTV MALL

使用selenium從hktvmall取得產品目錄 (Beautifulsoup 無法取得 ajax 架構的資料)

In [12]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
import time

In [13]:
# selenium init (firefox)
indexPage = webdriver.Firefox()
url = "https://www.hktvmall.com/hktv/zh/main/%E7%B6%AD%E7%89%B9%E5%81%A5%E9%9D%88/s/H0303001?q=::storeH0303001:"
indexPage.implicitly_wait(20)
indexPage.get(url)
time.sleep(2)

In [14]:
# RemoveAds
try: 
    indexPage.execute_script('document.getElementsByClassName("CrazyAd")[0].remove();')
except:
    1

# Grab Product
productDiv = indexPage.find_elements(By.CLASS_NAME, "product-brief")

In [15]:
# Grab Product titles, Prices and links
titles = []
prices = []
links = []
for product in productDiv:
    titles.append(product.find_element(By.CLASS_NAME, "brand-product-name").text)
    prices.append(product.find_element(By.CLASS_NAME, "price").text)
    links.append(product.find_element(By.CSS_SELECTOR, '[href^=main]').get_attribute('href'))

In [16]:
# Write into productDF
productDF = pd.DataFrame({"ProductName":titles, "Price":prices, "Link":links, "AvgStars":None, "Star5":None, "Star4":None, "Star3":None, "Star2":None, "Star1":None})

In [17]:
# Remove non product data
productDF = productDF.drop(productDF[(productDF['ProductName']=='') & (productDF['Price']=='')].index).reset_index(drop=1)

進入產品URL, 取得用戶評價

In [18]:
reviews = pd.DataFrame({"Link":[], "Comment":[], "Stars":[]})
for i, link in enumerate(productDF['Link']):
    try:
        # init selenium (firefox)
        productPage = webdriver.Firefox()
        productPage.implicitly_wait(20)
        productPage.get(link+"?scrollTo=reviewTab")
        time.sleep(2)

        # RemoveAds
        try: 
            productPage.execute_script('document.getElementsByClassName("CrazyAd")[0].remove();')
        except:
            1

        # Append ProductDF stars
        productDF["AvgStars"].loc[i] = productPage.find_element(By.CSS_SELECTOR, 'label.title-text').text
        productDF["Star5"].loc[i] = productPage.find_element(By.CSS_SELECTOR, 'div[data-tabname="star5"]').text
        productDF["Star4"].loc[i] = productPage.find_element(By.CSS_SELECTOR, 'div[data-tabname="star4"]').text
        productDF["Star3"].loc[i] = productPage.find_element(By.CSS_SELECTOR, 'div[data-tabname="star3"]').text
        productDF["Star2"].loc[i] = productPage.find_element(By.CSS_SELECTOR, 'div[data-tabname="star2"]').text
        productDF["Star1"].loc[i] = productPage.find_element(By.CSS_SELECTOR, 'div[data-tabname="star1"]').text

        for star in range(5,0,-1):
            comments = []            
            try:
                # Simulate User click on the [5~1]stars reviews
                productPage.find_element(By.CSS_SELECTOR, f'div[data-tabname="star{star}"]').click()
                
                # Grab how many comments are there
                reviewNo = re.findall(r"\(\d+\)", productPage.find_element(By.CSS_SELECTOR, f'div[data-tabname="star{star}"]').text)[0]
                reviewNo = re.sub(r'\(', '', reviewNo)
                reviewNo = re.sub(r'\)', '', reviewNo)
                reviewNo = int(reviewNo)

                # Grab first page comments
                for reviewi in productPage.find_elements(By.CSS_SELECTOR, 'div.review-title'):
                    comments.append(reviewi.text)

                # Since 1 page show 10 data, if the data%10==0 do 1 less click on "next page"
                if (reviewNo%10)==0:
                    reviewNo -= 1

                # Grab next pages comments
                for pageNo in range(int(reviewNo/10)):
                    productPage.find_element(By.CSS_SELECTOR, 'a.next-btn').click()
                    for commenti in productPage.find_elements(By.CSS_SELECTOR, 'div.review-title'):
                        comments.append(commenti.text)
                reviews = pd.concat([reviews, pd.DataFrame({"Link":link, "Comment":comments, "Stars":star})])
            except:
                # No comment on this star
                1
        # close firefox
        productPage.close()
    except:
        print(f'{link} No Comment or Connection Error')
        # close firefox
        productPage.close()

https://www.hktvmall.com/hktv/zh/main/%E7%B6%AD%E7%89%B9%E5%81%A5%E9%9D%88/s/main/%E7%B6%AD%E7%89%B9%E5%81%A5%E9%9D%88/s/H0303001/%E8%AD%B7%E7%90%86%E4%BF%9D%E5%81%A5/%E8%AD%B7%E7%90%86%E4%BF%9D%E5%81%A5/%E4%BF%9D%E5%81%A5%E7%94%A2%E5%93%81/%E9%9D%88%E8%8A%9D-%E5%86%AC%E8%9F%B2%E5%A4%8F%E8%8D%89-%E4%BA%BA%E8%94%98/%E5%8A%A0%E5%BC%B7%E9%85%8D%E6%96%B9%E7%9F%A5%E9%9F%B3%E8%9F%B2%E8%8D%89%E8%8F%8C%E7%B5%B2%E9%AB%94%E8%86%A0%E5%9B%8A-%E5%A2%9E%E5%BC%B7%E5%85%8D%E7%96%AB%E5%8A%9B%E6%94%B9%E5%96%84%E9%BC%BB%E6%95%8F-60%E7%B2%92/p/H0303001_S_FYY016 No Comment or Connection Error
https://www.hktvmall.com/hktv/zh/main/%E7%B6%AD%E7%89%B9%E5%81%A5%E9%9D%88/s/main/%E7%B6%AD%E7%89%B9%E5%81%A5%E9%9D%88/s/H0303001/%E8%AD%B7%E7%90%86%E4%BF%9D%E5%81%A5/%E8%AD%B7%E7%90%86%E4%BF%9D%E5%81%A5/%E4%BF%9D%E5%81%A5%E7%94%A2%E5%93%81/%E7%9C%BC%E6%99%B4%E5%81%A5%E5%BA%B7/%E7%9B%AE%E6%B8%85%E7%B4%A0-%E5%BC%B7%E6%95%88%E7%89%88-%E8%AD%B7%E7%9C%BC%E6%98%8E%E7%9B%AE%E8%97%8D%E8%8E%93%E7%B2%BE%E8%8F%AF%E5%8F%8A%E8%91

In [ ]:
# close firefox
indexPage.close()

#### Transfer to CSV

In [ ]:
# productDF.to_csv('products.csv')

In [ ]:
# reviews.to_csv('comment.csv')

# Transform

#### Basic data transform of productDF and reviews

In [ ]:
def stringInBracket(x):
    if (pd.isna(x)) | (x=='') :
        y = 0
    else:
        y = re.findall(r'\(\d+\)', x)[0]
        y = re.sub(r'\(', '', y)
        y = re.sub(r'\)', '', y)
        y = int(y)
    return y
for i in range(1,6):
    productDF[f'Star{i}'] = productDF[f'Star{i}'].apply(lambda x: stringInBracket(x))

In [ ]:
productDF["AvgStars"] = productDF["AvgStars"].str.replace('/5', '')

In [ ]:
productDF["AvgStars"] = productDF["AvgStars"].fillna(0).astype(float)

In [ ]:
productDF['Price'] = productDF['Price'].str.replace(r'$ ', "")
productDF['Price'] = productDF['Price'].str.replace(r',', "")
productDF['Price'] = productDF['Price'].astype(float)

In [ ]:
reviews['Stars'] = reviews['Stars'].astype(int)
reviews = reviews.reset_index(drop=1)

In [ ]:
reviews['Comment'] = reviews['Comment'].fillna('')

#### Add keywordsDF

In [ ]:
# 中文keywords modules
import jieba
import jieba.analyse
from collections import Counter
dictfile = "./dict.txt"  # 字典檔
stopfile = "./stopwords.txt"  # stopwords
jieba.set_dictionary(dictfile)
jieba.analyse.set_stop_words(stopfile)

In [ ]:
# 將所有comments混合成單一string
allcomments = ""
for text in reviews['Comment']:
     allcomments+=text

# 取600個keywords
tags = jieba.analyse.extract_tags(allcomments, topK=600) 

freq = {}
# 計算 frequency
seg_list = jieba.lcut(allcomments, cut_all=False)
dictionary = Counter(seg_list)
for ele in dictionary:
    if ele in tags:
        freq[ele] = dictionary[ele]

In [ ]:
#寫入 keywordsDF
keywordsDF = pd.DataFrame()
for kw, ct in freq.items():
    keywordsDF = pd.concat([keywordsDF, pd.DataFrame({'keywords':[kw], 'freq':[ct]})])

In [ ]:
keywordsDF = keywordsDF.set_index('keywords')

In [ ]:
reviews['keywords'] = ''
for keyword in keywordsDF.index:
    # print(keyword)
    for i in reviews[reviews['Comment'].str.contains(keyword)].index:
        reviews.loc[i, 'keywords'] += f"{keyword}, "
keywordsIncludedDF = pd.DataFrame({'included_keywords':reviews['keywords']})

#### Transfer to CSV

In [ ]:
# keywordsDF.to_csv('keywords.csv')

In [ ]:
# keywordsIncludedDF.to_csv('commentIncludeKeywords')

# Load to SQL

In [ ]:
from sqlalchemy import create_engine
import constrant # Save your sql connection data into constrant.py
user = constrant.user
password = constrant.password
hostname = constrant.hostname
database_name = 'vitagreen'
schema_name = 'vitagreen'
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{hostname}/{database_name}')

#### productDF to postgreSQL("vitagreen"."Products")

In [ ]:
# productDF.to_csv('productDF_transformed.csv')
connection = engine.raw_connection()
cursor = connection.cursor()
for index, row in productDF.iterrows():
     sqlMsg = f'INSERT INTO "vitagreen"."Products" (index, product_name, price, link, avg_stars, stars5, stars4, stars3, stars2, stars1) \
          VALUES ({index}, \'{row.ProductName}\', {row.Price}, \'{row.Link}\', {row.AvgStars}, {row.Star5}, {row.Star4}, {row.Star3}, {row.Star2}, {row.Star1})'
     cursor.execute(sqlMsg)
connection.commit()
cursor.close()

#### reviews to postgreSQL("vitagreen"."Comments")

In [ ]:
# reviews.to_csv('comment_transformed.csv')
connection = engine.raw_connection()
cursor = connection.cursor()
for index, row in reviews.iterrows():
     sqlMsg = 'INSERT INTO "vitagreen"."Comments" (index, link, comment, stars) \
          VALUES ({0}, \'{1}\', \'{2}\', {3})'.format(index, row.Link, re.sub(r"\'", "\'\'", row.Comment), row.Stars)
     cursor.execute(sqlMsg)
connection.commit()
cursor.close()

#### keywordsDF to postgreSQL("vitagreen"."keywords")

In [ ]:
keywordsDF.to_sql('keywords', con=engine, schema='vitagreen', if_exists='replace')

#### keywordsIncludedDF to postgreSQL("vitagreen"."included_keywords")

In [ ]:
keywordsIncludedDF.to_sql('included_keywords', con=engine, schema='vitagreen', if_exists='replace')